In [ ]:
# Streamlit adaptation of your gravity model simulation

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.table import Table

st.set_page_config(layout="wide")
st.title("Anderson & van Wincoop Gravity Model Simulation")

# Sidebar Controls
st.sidebar.header("GDP Parameters")
Y_orange = st.sidebar.slider("GDP Orange", 1, 100, 30)
Y_green = st.sidebar.slider("GDP Green", 1, 100, 30)
Y_row = st.sidebar.slider("GDP ROW", 10, 300, 90)

st.sidebar.header("Country X-Positions")
orange_x = st.sidebar.slider("Orange X", 2.0, 10.0, 2.4)
green_x = st.sidebar.slider("Green X", 2.0, 10.0, 8.0)
row_x = st.sidebar.slider("ROW X", 2.0, 10.0, 8.0)

st.sidebar.header("Model Settings")
seed = st.sidebar.slider("Seed", 0, 100, 60)
sigma = st.sidebar.slider("Sigma", 2, 10, 5)
dispersion = st.sidebar.slider("Dispersion", 0.5, 2.5, 1.5)
tau_mult = st.sidebar.slider("Tau Multiplier", 0.0, 2.5, 1.0)
asymmetric_trade = st.sidebar.slider("Tau Asymmetry", -1.0, 1.0, 0.0)

st.sidebar.header("Display Settings")
line_width = st.sidebar.slider("Line Width", 0.25, 4.0, 1.0)
show_mr = st.sidebar.checkbox("Show MR", False)
show_gdp_labels = st.sidebar.checkbox("Show GDP", False)
table_type = st.sidebar.selectbox("Table Type", ['none', 'flows', 'tau'])

np.random.seed(seed)

# GDP & Positions
Y_k, Y_l, Y_m, Y_n = np.random.dirichlet([1, 1, 1, 1]) * Y_row
gdps = [Y_orange, Y_green, Y_k, Y_l, Y_m, Y_n]
Y_i = np.array(gdps)
Y_w = Y_i.sum()

pos_orange = np.array([orange_x, 3.25])
pos_green = np.array([green_x, 3.25]) + np.random.uniform(-1, 1, size=2) * dispersion
row_positions = [np.array([row_x, 3.25]) + np.random.uniform(-1, 1, size=2) * dispersion for _ in range(4)]
positions = [pos_orange, pos_green] + row_positions
colors = ['darkorange', 'lightgreen'] + ['skyblue'] * 4

# Tau matrix
N = len(positions)
tau = np.ones((N, N))
for i in range(N):
    for j in range(N):
        if i != j:
            d_ij = np.linalg.norm(positions[i] - positions[j])
            direction = np.sign(positions[j][0] - positions[i][0])
            asym_factor = np.clip(1 + asymmetric_trade * direction / 2, 0.5, 1)
            tau[i, j] = (1 + d_ij * tau_mult / 4) * asym_factor

def compute_gravity_flows_optimized(Y_i, tau, sigma, tol=5e-3, max_iter=50):
    N = len(Y_i)
    Y_w = np.sum(Y_i)
    Y_share = Y_i / Y_w
    Pi = np.ones(N)
    P = np.ones(N)
    epsilon = 1e-9
    for _ in range(max_iter):
        Pi_prev = Pi.copy()
        P_prev = P.copy()
        for j in range(N):
            ratio = tau[:, j] / np.clip(Pi, epsilon, None)
            P[j] = np.power(np.sum(Y_share * np.clip(ratio, epsilon, None) ** (1 - sigma)), 1 / (1 - sigma))
        for i in range(N):
            ratio = tau[i, :] / np.clip(P, epsilon, None)
            Pi[i] = np.power(np.sum(Y_share * np.clip(ratio, epsilon, None) ** (1 - sigma)), 1 / (1 - sigma))
        if np.max(np.abs(P - P_prev)) < tol and np.max(np.abs(Pi - Pi_prev)) < tol:
            break
    numerator = np.outer(Y_i, Y_i) / Y_w
    denominator = np.outer(Pi, P)
    flows = numerator * (tau / np.clip(denominator, epsilon, None)) ** (1 - sigma)
    return flows, Pi, P

flows, Pi, P = compute_gravity_flows_optimized(Y_i, tau, sigma)
bilateral_trade = flows + flows.T

fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, 12)
ax.set_ylim(0, 6.5)
ax.set_aspect('equal')
ax.axis('off')

circle_labels = ['o', 'g', 'a', 'b', 'c', 'd']
def r(Y): return 0.15 + 0.02 * np.sqrt(Y)
for idx, (pos, gdp, color) in enumerate(zip(positions, gdps, colors)):
    ax.add_patch(plt.Circle(pos, r(gdp), color=color, alpha=0.8))
    ax.text(pos[0], pos[1], circle_labels[idx], ha='center', va='center', fontsize=12, color='black')

for i in range(len(positions)):
    for j in range(i + 1, len(positions)):
        lw_ij = flows[i, j] * line_width
        alpha_ij = min(1.0, 0.0005 + lw_ij / 4)
        ax.annotate('', xy=positions[j], xytext=positions[i],
                    arrowprops=dict(arrowstyle='<->', lw=lw_ij, color='black', alpha=alpha_ij))

if show_mr or show_gdp_labels:
    for i, pos in enumerate(positions):
        label_parts = []
        if show_mr:
            label_parts.append(f"$\\Pi_{circle_labels[i]}$: {Pi[i]:.2f}, $P_{circle_labels[i]}$: {P[i]:.2f}")
        if show_gdp_labels:
            label_parts.append(f"$Y_{circle_labels[i]}$: {gdps[i]:.0f}")
        label_text = ", ".join(label_parts)
        ax.text(pos[0], pos[1] + r(gdps[i]) + 0.05, label_text, ha='center', fontsize=10)

gravity_val = (gdps[0] * gdps[1]) / Y_w * (tau[0, 1] / (Pi[0] * P[1]))**(1 - sigma)
ax.text(0.05, 6.2,
        f"$X_{{og}} = \\left(\\frac{{Y_o Y_g}}{{Y_W}}\\right) \\cdot \\left(\\frac{{\\Pi_i P_j}}{{\\tau_{{ij}}}}\\right)^{{\\sigma-1}} = {gravity_val:.2f}$",
        fontsize=14, ha='left')

st.pyplot(fig)
